In [3]:
import numpy as np
import math
# import csv
import pandas as pd

np.seterr(all="raise")  # Error on overflow

# Initial Parameters
LOAD_DEAD = 8000  # kN
LOAD_LIVE = 2500  # kN
BEARINGPRESSURE_ALLOWABLE = 150  # kPa
FTG_COVER = 0.060  # m
COLUMN_WIDTH = 0.500  # m


class FoundationSizes:
    def __init__(self):
        FTG_LEN_MIN = (
            math.ceil(
                math.sqrt((LOAD_DEAD + LOAD_LIVE) / BEARINGPRESSURE_ALLOWABLE) / 0.05
            )
            * 0.05
        )
        FTG_LEN_MAX = FTG_LEN_MIN + 1
        FTG_LENS = np.round(
            np.arange(FTG_LEN_MIN, FTG_LEN_MAX + 0.001, 0.05, dtype=np.float32), 2
        )

        FTG_DPTH_MIN = (
            math.ceil(
                (
                    4
                    * math.sqrt(3570)
                    * math.sqrt(
                        (4760 * COLUMN_WIDTH**2) / 3
                        + (1 + (3 * BEARINGPRESSURE_ALLOWABLE) / 19040)
                        * (LOAD_DEAD + LOAD_LIVE)
                    )
                    - (9520 + 3 * BEARINGPRESSURE_ALLOWABLE) * COLUMN_WIDTH
                )
                / (19040 + 3 * BEARINGPRESSURE_ALLOWABLE)
                / 0.05
            )
            * 0.05
        )
        FTG_DPTH_MAX = FTG_DPTH_MIN * 2
        FTG_DPTHS = np.round(
            np.arange(FTG_DPTH_MIN, FTG_DPTH_MAX + 0.001, 0.05, dtype=np.float32), 2
        )

        FTG_CONC_STRENGTHS = np.array([20, 25, 32, 40, 50, 65], dtype=np.float32)

        FTG_REO_SIZES = np.round(
            np.array(
                [0.01, 0.012, 0.016, 0.02, 0.024, 0.028, 0.032, 0.036, 0.04],
                dtype=np.float32,
            ),
            3,
        )

        FTG_REO_CTS = np.unique(
            np.round(
                np.concatenate(
                    [
                        np.arange(0.1, 0.301, 0.025, dtype=np.float32),
                        np.arange(0.08, 0.301, 0.02, dtype=np.float32),
                    ]
                ),
                3,
            )
        )

        foundation_sizes = np.array(
            np.meshgrid(
                FTG_LENS, FTG_DPTHS, FTG_CONC_STRENGTHS, FTG_REO_SIZES, FTG_REO_CTS
            )
        ).T.reshape(-1, 5)

        # Instantiate the full array with additional empty columns
        self.sizes = np.zeros((foundation_sizes.shape[0], 20), dtype=np.float32)
        self.sizes[:, :5] = foundation_sizes

        self.FtgLength = self.sizes[:, 0]
        self.FtgDepth = self.sizes[:, 1]
        self.fc = self.sizes[:, 2]
        self.ReoSize = self.sizes[:, 3]
        self.ReoCts = self.sizes[:, 4]
        self.BPmax = self.sizes[:, 5]
        self.BP_Ratio = self.sizes[:, 6]
        self.Pult = self.sizes[:, 7]
        self.Dom = self.sizes[:, 8]
        self.BPult = self.sizes[:, 9]

        self.calc_BPmax()
        self.calc_Pult()
        self.calc_ratio_bearing()
        self.calc_Dom()
        self.calc_BPult()

    def calc_BPmax(self):
        self.BPmax[:] = (
            6 * self.FtgDepth * self.FtgLength**2 + LOAD_LIVE + LOAD_DEAD
        ) / (self.FtgLength**2)

    def calc_ratio_bearing(self):
        self.BP_Ratio[:] = BEARINGPRESSURE_ALLOWABLE / self.BPmax

    def calc_Pult(self):
        self.Pult[:] = (
            1.2 * (6 * self.FtgDepth * self.FtgLength**2 + LOAD_DEAD)
            + 1.5 * LOAD_LIVE
        )

    def calc_Dom(self):
        self.Dom[:] = self.FtgDepth - FTG_COVER - self.ReoSize / 2

    def calc_BPult(self):
        self.BPult[:] = self.Pult / self.FtgLength**2

    def print_raw_array(self, num_rows=5):
        print(self.sizes[0, :])


    def print_foundation_sizes(self, num_rows=6):
        column_headers = [
            attr
            for attr in self.__dict__
            if isinstance(getattr(self, attr), np.ndarray)
        ]
        pd.set_option("display.float_format", lambda x: "%.3f" % x)
        num_columns = len(column_headers)
        
        # Calculate the step size based on the total number of rows
        total_rows = self.sizes.shape[0]
        step = max(1, total_rows // (num_rows - 1))
        
        # Select rows at regular intervals
        selected_rows = np.arange(0, total_rows, step)
        if len(selected_rows) < num_rows:
            selected_rows = np.append(selected_rows, total_rows - 1)
        elif len(selected_rows) > num_rows:
            selected_rows = selected_rows[:num_rows]
        
        df = pd.DataFrame(
            self.sizes[selected_rows, :num_columns-1], 
            columns=column_headers[:num_columns-1],
            index=selected_rows
        )
        print(df.to_string(header=True))


In [4]:
# Create an instance of the FoundationSizes class
foundation = FoundationSizes()
foundation.print_foundation_sizes()
foundation.sizes.shape

        sizes  FtgLength  FtgDepth    fc  ReoSize  ReoCts  BPmax  BP_Ratio  Pult     Dom
0       8.400      1.050    20.000 0.010    0.080 155.110  0.967 13883.435 0.985 196.761
175348  9.200      1.450    32.000 0.028    0.125 132.755  1.130 14233.643 1.376 168.167
350696 10.000      1.850    50.000 0.012    0.175 116.100  1.292 14682.000 1.784 146.820
526044  8.800      1.150    25.000 0.036    0.220 142.489  1.053 13991.203 1.072 180.672
701392  9.600      1.550    40.000 0.020    0.260 123.232  1.217 14378.506 1.480 156.017
876740 10.400      1.950    65.000 0.040    0.300 108.778  1.379 14868.566 1.870 137.468


(876744, 20)